# TensorFlow MNIST - Basic - Solution
The MNIST database (Mixed National Institute of Standards and Technology database) is a large database of handwritten digits that is commonly used for training various image processing systems. For some backgrund on the data, see [Wikipedia](https://en.wikipedia.org/wiki/MNIST_database).

This notebook will guide you through implementing a basic TensorFlow feed forward neural network to classify handwritten digits.

## Data
Each image consists of $28 \times 28$ pixels. We will treat each pixel as a feature, therefore we have 784 individual features. In the first cell below we will first download the data, that might take a second but only has to be done once. In the next cell we then visualize some samples from the data so you have an idea what you are dealing with.

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

In [ ]:
%matplotlib notebook
from matplotlib import pyplot as plt
import numpy as np
from numpy.random import choice

fig = plt.figure('Available Datasets')
for i, sample in enumerate(choice(len(mnist.train.images), 9, replace=False)):
    image = mnist.train.images[sample].reshape((28,28))
    label = mnist.train.labels[sample]
    axis = plt.subplot(331+i)
    axis.set_title(np.argmax(label))
    axis.imshow(image, cmap='gray')
fig.canvas.draw()

## Model
Each of the data samples consists of 784 features and there is a total of 10 digits. Remember that in TensorFlow the placeholders need to reflect that we want to handle the data in batches -- therefore the input and output are not one but two dimensional. The calculations for the output predictions are already given here. Feel free to try out different activation functions found in the TensorFlow [API Documentation](https://www.tensorflow.org/versions/r0.9/api_docs/python/nn.html#neural-network). For details on why we use softmax here checkout the [official tutorial](https://www.tensorflow.org/versions/r0.9/tutorials/mnist/beginners/index.html#softmax-regressions).

For learning we are going to use a gradient descent optimizer already provided by TensorFlow. A well proven error measure is [cross entropy](https://en.wikipedia.org/wiki/Cross_entropy):

$$-\sum(targets * log(predictions))$$

TensorFlow provides functions for all of those calculations: `tf.reduce_sum` and `tf.log`.

In [ ]:
x = tf.placeholder(tf.float32, shape=(None,784), name='input')
t = tf.placeholder(tf.float32, shape=(None, 10), name='target')

W = tf.Variable(tf.zeros((784,10)), name='weights')
b = tf.Variable(tf.zeros((10)), name='biases')

y = tf.nn.softmax(tf.matmul(x, W) + b, name='predictions')

cross_entropy = -tf.reduce_sum(t*tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

## Training & Testing
THe training and testing code again is given. Take note of the correct prediction checking and accuracy measurement: The target vectors always contain nine zeros and a single one. Using `tf.argmax` we retrieve the index of the single one from the predictions and from the targets and compare them with each other. In the accuracy measure we then cast this boolean result to a float (results in either `0.` or `1.` and calculate the mean over the whole batch using `tf.reduce_mean`.

In [ ]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(t,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess = tf.Session()
sess.run(tf.initialize_all_variables())

for i in range(1000):
    batch_x, batch_t = mnist.train.next_batch(100)
    sess.run(train_step, {x: batch_x, t: batch_t})

# Check the results.
print(sess.run(accuracy, {x: mnist.test.images, t: mnist.test.labels}))